# Ski Bearing Playground

- https://geoffboeing.com/2018/07/comparing-city-street-orientations/
- groomed v ungroomed
- https://github.com/tilezen/joerd
- trail orientation v aspect
- weight by elevation descent
- max slope v difficulty by region
- https://osmnx.readthedocs.io/en/stable/internals-reference.html-osmnx-bearing-module
- https://stackoverflow.com/questions/54873868/python-calculate-bearing-between-two-lat-long
- https://github.com/gboeing/osmnx/blob/cabdbb246dc14095f5d8e8b8ead38f9649dae2c7/osmnx/bearing.py#L18


In [1]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_formats = ['svg']

In [2]:
import plotly.express as px
import polars as pl

import ski_bearings.analyze
import ski_bearings.plot
import ski_bearings.utils

In [4]:
ski_areas_pl = ski_bearings.analyze.load_ski_areas_pl()
bearing_dists_pl = ski_bearings.analyze.load_bearing_distribution_pl()

In [ ]:
import reactable

from ski_bearings.analyze import get_ski_area_reactable

reactable.embed_css()

get_ski_area_reactable()

In [4]:
new_england_ski_area_df = (
    ski_areas_pl.filter(pl.col("status") == "operating")
    .filter(pl.col("ski_area_name").is_not_null())
    .filter(pl.col("location__localized__en__country") == "United States")
    .filter(
        pl.col("location__localized__en__region").is_in(
            ["New Hampshire", "Vermont", "Maine", "Massachusetts"]
        )
    )
    .filter(pl.col("run_count_filtered") > 0)
    .filter(pl.col("run_count_filtered") > 10)
)

In [ ]:
ski_bearings.plot.subplot_orientations(
    groups_pl=new_england_ski_area_df.filter(
        pl.col("location__localized__en__region") == "New Hampshire"
    ),
    grouping_col="ski_area_name",
    n_cols=5,
    free_y=True,
)

In [ ]:
ski_bearings.plot.subplot_orientations(
    groups_pl=new_england_ski_area_df.filter(
        pl.col("location__localized__en__region") == "Vermont"
    ),
    grouping_col="ski_area_name",
    n_cols=5,
    free_y=True,
)

In [ ]:
px.scatter_polar(
    data_frame=new_england_ski_area_df.rename(
        {"location__localized__en__region": "state"}
    ),
    r="mean_bearing_strength",
    theta="mean_bearing",
    size="combined_vertical",
    hover_name="ski_area_name",
    hover_data=["run_count_filtered", "run_count"],
    color="state",
    range_r=[0, 1],
    width=800,
    height=800,
)

In [ ]:
ski_bearings.analyze.bearing_dists_by_hemisphere().select(
    "hemisphere", "bearings"
).explode("bearings").unnest("bearings").filter(pl.col("num_bins") == 2)

In [ ]:
ski_areas_pl.get_column("status").value_counts()

In [ ]:
ski_areas_pl.get_column("runConvention").value_counts()